In [ ]:
# Cell 1: Upload the zipped dataset from your PC
# This will open a file browser dialog. Select your 'plastic_dataset.zip' file.
from google.colab import files
uploaded = files.upload()

In [ ]:
# Cell 2: Unzip the uploaded file

import os
import zipfile

# The name of your uploaded zip file
zip_file_name = 'Plastic waste 2.v3-v1.1.yolov8.zip'

# The directory where the contents will be unzipped.
# We'll use the file name to automatically create a folder.
data_path = f"/content/{zip_file_name.split('.')[0]}"

# Create the directory if it doesn't exist
os.makedirs(data_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(data_path)

print(f"Dataset successfully unzipped to {data_path}")

Dataset successfully unzipped to /content/Plastic waste 2


In [ ]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.5 MB/s eta 0:00:00


In [ ]:

from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Load a pre-trained YOLOv8n model
model = YOLO('yolov8n.pt')

In [ ]:
import os
print(os.listdir("/content"))


['.config', 'Plastic waste 2', 'Plastic waste 2.v3-v1.1.yolov8.zip', 'yolov8n.pt', 'sample_data']


In [ ]:
import zipfile

with zipfile.ZipFile("/content/Plastic waste 2.v3-v1.1.yolov8.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")


In [ ]:
import os
print(os.listdir("/content"))
print(os.listdir("/content/Plastic waste 2"))


['.config', 'README.roboflow.txt', 'data.yaml', 'Plastic waste 2', 'Plastic waste 2.v3-v1.1.yolov8.zip', 'train', 'valid', 'test', 'yolov8n.pt', 'README.dataset.txt', 'sample_data']
['README.roboflow.txt', 'data.yaml', 'train', 'valid', 'test', 'README.dataset.txt']


In [ ]:
import yaml

# Path to the dataset's own yaml (usually correct class names & nc are in there)
dataset_yaml = "/content/Plastic waste 2/data.yaml"
root_yaml = "/content/data.yaml"

# Load the dataset yaml
with open(dataset_yaml, "r") as f:
    data = yaml.safe_load(f)

# Fix the paths
data["train"] = "/content/Plastic waste 2/train/images"
data["val"]   = "/content/Plastic waste 2/valid/images"
data["test"]  = "/content/Plastic waste 2/test/images"

# Save to root data.yaml (this is the one YOLOv8 will use)
with open(root_yaml, "w") as f:
    yaml.dump(data, f, default_flow_style=False)

print("✅ data.yaml updated successfully!")


✅ data.yaml updated successfully!


In [ ]:
# Train the model with your dataset
results = model.train(data='data.yaml', epochs=50, imgsz=640, batch=16)

Ultralytics 8.3.192 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fals

In [ ]:
# Cell 6: Validate the trained model

# The model will automatically use the 'test' path from your data.yaml
metrics = model.val()

print("Validation Results:")
print(f"Mean Average Precision (mAP50-95): {metrics.results_dict['metrics/mAP50-95(B)']: .4f}")
print(f"Precision: {metrics.results_dict['metrics/precision(B)']: .4f}")
print(f"Recall: {metrics.results_dict['metrics/recall(B)']: .4f}")


Ultralytics 8.3.192 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 582.0±267.0 MB/s, size: 15.9 KB)
val: Scanning /content/Plastic waste 2/valid/labels.cache... 449 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 449/449 958393.1it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 29/29 6.1it/s 4.7s
                   all        449        558      0.867      0.873      0.926      0.746
       Non stretch bag         65         76       0.94      0.832      0.925      0.796
                OTHERS         13         26       0.78      0.682      0.788      0.497
            PET bottle         72         78      0.858      0.962      0.971      0.783
      PET color bottle         35         43      0.852      0.977      0.977      0.818
      PET-PS packaging         46      

In [ ]:
import os

for root, dirs, files in os.walk("runs"):
    for file in files:
        if file.endswith(".pt"):
            print(os.path.join(root, file))


runs/detect/train/weights/last.pt
runs/detect/train/weights/best.pt


In [ ]:
from google.colab import files
files.download('runs/detect/train/weights/best.pt')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>